In [95]:
import numpy as np
import pandas as pd
import math
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

In [96]:
#dataset
PATH_X_TRAIN='./wili-2018/x_train.txt'
PATH_Y_TRAIN='./wili-2018/y_train.txt'
PATH_X_TEST='./wili-2018/x_test.txt'
PATH_Y_TEST='./wili-2018/y_test.txt'

#model 
INPUT_SIZE=1000

In [97]:
#read dataset
def read_txt(path):
    handle = open(path, "r", encoding='utf8')
    df = pd.DataFrame(handle.readlines())
    handle.close()
    return df

x_train=read_txt(PATH_X_TRAIN)
y_train=read_txt(PATH_Y_TRAIN)
x_test=read_txt(PATH_X_TEST)
y_test=read_txt(PATH_Y_TEST)

KeyboardInterrupt: 

In [ ]:
#texts preprocessing
texts_train=x_train.values
texts_test=x_test.values
texts_train=[s[0].replace('\n','').lower() for s in texts_train]
texts_test=[s[0].replace('\n','').lower() for s in texts_test]
tokenizer=Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tokenizer.fit_on_texts(texts_train)

#sequences
texts_train=tokenizer.texts_to_sequences(texts_train)
texts_test=tokenizer.texts_to_sequences(texts_test)

texts_train=pad_sequences(texts_train, maxlen=INPUT_SIZE, padding='post')
texts_train=np.array(texts_train)
texts_test=pad_sequences(texts_test, maxlen=INPUT_SIZE, padding='post')
texts_test=np.array(texts_test)

In [ ]:
#classes preprocessing
classes_train=y_train.values
classes_test=y_test.values

#dictionary for languages
classes = np.unique(np.array(classes_train))
nums=np.arange(len(classes))
d = dict(zip(classes,nums))

classes_train=[d[c[0]] for c in classes_train] 
classes_test=[d[c[0]] for c in classes_test] 
classes_train=to_categorical(classes_train)
classes_test=to_categorical(classes_test)

In [ ]:
voc_size=len(tokenizer.word_index)
voc_size

In [ ]:
#embeddings
embeddings_weights=[]
embeddings_weights.append(np.zeros(voc_size))


for char, i in tokenizer.word_index.items():
    onehot=np.zeros(voc_size)
    onehot[i-1]=1
    embeddings_weights.append(onehot)
embeddings_weights=np.array(embeddings_weights)

In [88]:
embedding_layer=Embedding(voc_size+1,
                         voc_size,
                         input_length=1000,
                         weights=[embeddings_weights])

In [89]:
conv_layers=[[256, 7, 3],
            [256, 7, 3],
            [256, 7, 3],
            [256, 3, -1],
            [256, 3, -1],
            [256, 3, -1],
            [256, 3, 3]]

fully_connected_layers=[1024, 1024]
num_of_classes=len(classes)
dropout=0.5
optimizer='adam'
loss='categorical_crossentropy'

In [91]:
inputs=Input(shape=(1000,), name='input', dtype='int64')
x=embedding_layer(inputs)
for filter_num, filter_size, pooling_size in conv_layers:
    x=Conv1D(filter_num, filter_size)(x)
    x=Activation('relu')(x)
    if pooling_size!=-1:
        x=MaxPooling1D(pool_size=pooling_size)(x)
x=Flatten()(x)
for dense_size in fully_connected_layers:
    x=Dense(dense_size, activation='softmax')(x)
    x=Dropout(dropout)(x)
predictions=Dense(num_of_classes, activation='softmax')(x)
model=Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 10490)       110050590 
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 994, 256)          18798336  
_________________________________________________________________
activation_8 (Activation)    (None, 994, 256)          0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 331, 256)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 325, 256)          459008    
_________________________________________________________________
activation_9 (Activation)    (None, 325, 256)          0         
__________

In [92]:
model.fit(texts_train, classes_train, 
          validation_data = (texts_test, classes_test),
         batch_size=128,
         epochs=10,
         verbose=2)

Instructions for updating:
Use tf.cast instead.


C:\Users\Annie\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:107: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 110050590 elements. This may consume a large amount of memory.
  num_elements)


Train on 117500 samples, validate on 117500 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[10491,10490] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu
	 [[{{node training/Adam/mul_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
